In [12]:
%run Loading_data.ipynb

In [13]:
import torch
from torch import nn
from torch.nn.utils.rnn import pad_sequence
import torch.nn.functional as F

In [14]:
batch = []
target = []
for j in range(500):
    tensor_x = []
    for i in range(40):
        tensor_x.append(torch.tensor(x_train[i+j*40], dtype=torch.long))
    tensor_pad = pad_sequence(tensor_x, batch_first=True, padding_value=0)
    batch.append(tensor_pad)
    target.append(torch.tensor(y_train[0+j*40:40+j*40]))

In [15]:
v_batch = []
v_target = []
for j in range(125):
    tensor_x = []
    for i in range(40):
        tensor_x.append(torch.tensor(x_train[i+j*40], dtype=torch.long))
    tensor_pad = pad_sequence(tensor_x, batch_first=True, padding_value=0)
    v_batch.append(tensor_pad)
    v_target.append(torch.tensor(y_train[0+j*40:40+j*40]))

In [16]:
class Net(nn.Module):

    def __init__(self):
        super(Net, self).__init__()
        # 1 input image channel, 16 output channels, 3x3 square convolution
        # kernel
        self.layer1 = nn.Embedding(len(i2w),300)
        # an affine operation: y = Wx + b
        self.fc1 = nn.Linear(300,300)  
        self.fc2 = nn.Linear(300,2)

    def forward(self, x):
        x = self.layer1(x)
        x = torch.max(F.relu(self.fc1(x)),1)
        x = self.fc2(x.values)
        return x

net = Net()
print(net)

Net(
  (layer1): Embedding(99430, 300)
  (fc1): Linear(in_features=300, out_features=300, bias=True)
  (fc2): Linear(in_features=300, out_features=2, bias=True)
)


In [17]:
import torch.optim as optim
optimizer = optim.Adam(net.parameters(),betas=(0.9, 0.999), lr=0.0005)

In [18]:
for epoch in range(1): 
    for j in range(500):
        outputs = net(batch[j])
        optimizer.zero_grad()
        loss = torch.nn.functional.cross_entropy(F.softmax(outputs,1),target[j])
        loss.backward()
        optimizer.step()
        print(loss.item())

print('Finished Training')

0.7001241445541382
0.7150899171829224
0.6627795100212097
0.6540238857269287
0.6366941928863525
0.657733142375946
0.7591081857681274
0.8538466691970825
0.6231381893157959
0.5254716277122498
0.7184483408927917
0.7147964239120483
0.7689799070358276
0.7658973932266235
0.6440830826759338
0.682274580001831
0.6421079635620117
0.6639162302017212
0.6613119840621948
0.675513744354248
0.6635802388191223
0.6465173363685608
0.6434848308563232
0.6351696252822876
0.5986979007720947
0.7097248435020447
0.684760570526123
0.6138460040092468
0.6622329950332642
0.7001656293869019
0.6496512293815613
0.642638087272644
0.6129118800163269
0.6189146041870117
0.5985599756240845
0.6103549003601074
0.5963338613510132
0.6092076897621155
0.599199652671814
0.5961412191390991
0.6233580708503723
0.6234766244888306
0.5956727266311646
0.5467987656593323
0.5975139737129211
0.6005131602287292
0.6302342414855957
0.5522904396057129
0.5967479944229126
0.6365700960159302
0.5535088181495667
0.5972403287887573
0.5806043148040771

0.5614744424819946
0.46035805344581604
0.49954819679260254
0.47527962923049927
0.5126016139984131
0.4267142415046692
0.4807652533054352
0.536629855632782
0.4788537919521332
0.5083392858505249
0.47262391448020935
0.4723193645477295
0.49001622200012207
0.43655553460121155
0.5728859305381775
0.5246618390083313
0.5135493874549866
0.533228874206543
0.39447423815727234
0.546818196773529
0.5578999519348145
0.5708019137382507
0.5053207874298096
0.4773697257041931
0.4232805371284485
0.5452023148536682
0.5146154761314392
0.4525693953037262
0.45708179473876953
0.41224271059036255
0.4760531783103943
0.46670103073120117
0.4741618037223816
0.52752286195755
0.4545159339904785
0.4910198748111725
0.44732117652893066
0.4337555468082428
0.49452081322669983
0.48896822333335876
0.47923341393470764
0.4217609465122223
0.39030465483665466
0.47402334213256836
0.4940546154975891
0.45064935088157654
0.4356105327606201
0.43621787428855896
0.386301189661026
0.47963976860046387
0.49094367027282715
0.511131286621093

In [19]:
correct = 0
total = 0
with torch.no_grad():
    for j in range(125):
        outputs = net(v_batch[j])
        _, predicted = torch.max(outputs.data, 1)
        total += v_target[j].size(0)
        correct += (predicted == v_target[j]).sum().item()

print('Accuracy of the network on the 5000 test: %d %%' % (
    100 * correct / total))

Accuracy of the network on the 5000 test images: 87 %
